In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f1e56a2-4fe7-4a2e-abf3-05aaf746e9d2;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
downloading https://repo1.maven.org/maven2/com/databricks/spark-xml_2.12/0.14.0/spark-xml_2.12-0.14.0.jar ...
	[SUCCESSFUL ] com.databricks#spark-xml_2.12;0.14.0!spark-xml_2.12.jar (24ms)
downloading https://repo1.maven.org/maven2/commons-io/commons-io/2.8.0/commons-io-2.8.0.jar ...
	[SUCCESSFUL ] commons-io#commons-io;2.8.0!commons-io.jar (18ms)
downloading https://repo1.maven.org/maven2/org/glassfish/jaxb/txw2/2.3.4/txw2-2.3.4.jar ...
	[SUCCESSFUL ] org.glassfish.jaxb#txw2;2.3.4!txw2.jar (14ms)
downloading

In [17]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import desc

In [5]:
fp="gs://hw2_bucketwb/notebooks/jupyter/task2samllfull.csv"
df = spark.read.options(header='False', inferSchema=True, delimiter='\t').csv(fp)
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [7]:
df2 = df.withColumnRenamed('_c0','title').withColumnRenamed('_c1','link')
df2.show(10)

+--------------------+--------------------+
|               title|                link|
+--------------------+--------------------+
|industrial revolu...|interchangeable p...|
|industrial revolu...|interchangeable p...|
|industrial revolu...|interchangeable p...|
|industrial revolu...|interchangeable p...|
|industrial revolu...|interchangeable p...|
|industrial revolu...|interchangeable p...|
|industrial revolu...|iron and steel in...|
|industrial revolu...|       islamic world|
|industrial revolu...| italian city-states|
|industrial revolu...|       ivy pinchbeck|
+--------------------+--------------------+
only showing top 10 rows



In [13]:
# convert DataFrame to RDD
title_links = df2.rdd.map(lambda item:(item['title'], item['link'])).distinct().groupByKey().cache()
title_rank = title_links.map(lambda title: (title[0], 1.0))
# title_rank.take(1)

[('industrial workers of the world', 1.0)]

In [14]:
# title_links.join(title_rank).take(1)

[('infantry fighting vehicle',
  (<pyspark.resultiterable.ResultIterable at 0x7f53aade4eb0>, 1.0))]

In [16]:
n = 10
for i in range(n):
    contributions = title_links.join(title_rank)\
    .flatMap(lambda title_links_rank:\
    [(link, title_links_rank[1][1] / len(title_links_rank[1][0])) for link in title_links_rank[1][0]])
    
    title_rank = contributions.reduceByKey(lambda cont1, cont2: cont1 + cont2)\
    .mapValues(lambda cont: 0.15 + 0.85 * cont)

In [18]:
schema_df3 = StructType([StructField("article", StringType(), True), StructField("rank", FloatType(), True)])

# convert RDD to DataFrame
df3 = spark.createDataFrame(title_rank, schema_df3)
df4 = df3.sort(desc("rank")).limit(10)
df4.show(10)

+-----------------+---------+
|          article|     rank|
+-----------------+---------+
|  catholic church|13.110594|
|           france| 9.076213|
|    new york city| 7.034899|
|         buddhism|6.7305155|
|            islam|5.1928144|
|       philosophy| 5.109292|
|      linguistics|  4.19185|
|     cryptography| 3.774827|
|   climate change| 3.680087|
|personal computer|3.5385349|
+-----------------+---------+



In [ ]:
df4.repartition(1).\
write.mode("overwrite").csv("gs://hw2_bucketwb/notebooks/jupyter/task3.csv",header=False,sep='\t')